<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020_0926PyTorch_nn_tutorial_again.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- source: https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/5ddab57bb7482fbcc76722617dd47324/nn_tutorial.ipynb#scrollTo=1jkWbziDeGBV
- date: 2020-0926


In [1]:
%matplotlib inline


What is `torch.nn` *really*?
============================
by Jeremy Howard, `fast.ai <https://www.fast.ai>`_. Thanks to Rachel Thomas and Francisco Ingham.



PyTorch には，頻用されるクラスが用意されています。例えば，以下のようなものです:

- [torch.nn](https://pytorch.org/docs/stable/nn.html),
- [torch.optim](https://pytorch.org/docs/stable/optim.html),
- [Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset),
- [DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader),

これらがどのように動作するかを知ることが，ディープラーニングの理解を促進します。
実際にコードを書いてみること理解が深まります。

ここではそれぞれを使ったコードを書いてみましょう。


In [ ]:
#  日本語くずし字データセットをダウンロード
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz


In [ ]:
# ダウロードしたデータを読み込み
import numpy as np

def load(f):
    return np.load(f)['arr_0']

# Load the data
Xkm_train = load('kmnist-train-imgs.npz')
Xkm_test = load('kmnist-test-imgs.npz')
ykm_train = load('kmnist-train-labels.npz')
ykm_test = load('kmnist-test-labels.npz')

# Flatten images
n_samples = 2000
#x_train = x_train.reshape(-1, 784)[:n_samples]
#y_train = y_train[:n_samples]
#x_test = x_test.reshape(-1, 784)

ind2c =[c for c in 'おきすつなまはやれを']

!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
from matplotlib import pyplot

pyplot.imshow(Xkm_train[0], cmap="gray") # 確認のためデータを表示
#print(x_train.shape)

PyTorch uses ``torch.tensor``, rather than numpy arrays, so we need to
convert our data.



In [5]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (Xkm_train, ykm_train, Xkm_test, ykm_test)
)

# np.array と異なり PyTorch ではデフォルトの浮動小数点サイズが 32 なので変換しておく
x_train, x_valid = x_train.float(), x_valid.float()
y_train, y_valid = y_train.long(), y_valid.long()
#n, c, _ = x_train.shape

print(x_train.shape, y_train.min(), y_train.max())

torch.Size([60000, 28, 28]) tensor(0) tensor(9)


# 簡単なニューラルネットワーク

結合係数行列とバイアスを使って簡単なニューラルネットワークを作ってみましょう。
PyTorch には自動微分機能があり，`require_grad()` を使うことでどの変数を学習させるかを定義します。
定義するだけで，誤差逆伝播法に基づく学習は自動的に行なってくれます。

学習の直前に ``require_grad`` をセットする必要があります。
PyTorch では 後尾に``_``が付いた操作は，その場で変数が書き換わる（インライン操作）を意味します。


In [6]:
import math

n_neurons = 784
n_classes = 10
weights = torch.randn(n_neurons, n_classes) / math.sqrt(n_neurons)  # ザビエルの初期化
weights.requires_grad_()
bias = torch.zeros(n_classes, requires_grad=True)

In [7]:
# ソフトマックス関数の対数
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(X):
    return log_softmax(X.reshape(-1, n_neurons).mm(weights) + bias)


In the above, the ``@`` stands for the dot product operation. We will call
our function on one batch of data (in this case, 64 images).  This is
one *forward pass*.  Note that our predictions won't be any better than
random at this stage, since we start with random weights.



In [11]:
batch_size = 64  # batch size

X = x_train[0:batch_size] # a mini-batch from x
preds = model(X)  # predictions
print(preds[0], preds.shape)
#print(preds)

tensor([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       grad_fn=<SelectBackward>) torch.Size([64, 10])


As you see, the ``preds`` tensor contains not only the tensor values, but also a
gradient function. We'll use this later to do backprop.

Let's implement negative log-likelihood to use as the loss function
(again, we can just use standard Python):



In [12]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

Let's check our loss with our random model, so we can see if we improve
after a backprop pass later.



In [13]:
y = y_train[0:batch_size]
print(loss_func(preds, y))

tensor(inf, grad_fn=<NegBackward>)


Let's also implement a function to calculate the accuracy of our model.
For each prediction, if the index with the largest value matches the
target value, then the prediction was correct.



In [14]:
def accuracy(out, y):
    preds = torch.argmax(out, dim=1)
    return (preds == y).float().mean()

Let's check the accuracy of our random model, so we can see if our
accuracy improves as our loss improves.



In [15]:
print(accuracy(preds, y))

tensor(0.0469)


ニューラルネットワークの学習，訓練時には以下の手順になります:

1. ミニバッチサイズを決める
2. モデルを定める
3. 損失関数を計算する
4. ``loss.backward()`` で，結合係数のパラメータをモデルの勾配から更新する

We now use these gradients to update the weights and bias.  We do this
within the ``torch.no_grad()`` context manager, because we do not want these
actions to be recorded for our next calculation of the gradient.  You can read
more about how PyTorch's Autograd records operations
[here](https://pytorch.org/docs/stable/notes/autograd.html).

We then set the gradients to zero, so that we are ready for the next loop.
Otherwise, our gradients would record a running tally of all the operations that had happened (i.e. ``loss.backward()`` *adds* the gradients to whatever is already stored, rather than replacing them).

#### tip
You can use the standard python debugger to step through PyTorch    code, allowing you to check the various variable values at each step.
Uncomment ``set_trace()`` below to try it out.




In [47]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 5  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n_samples - 1) // batch_size + 1):
        # set_trace()
        start_i = i * batch_size
        end_i = start_i + batch_size
        X = x_train[start_i:end_i]
        y = y_train[start_i:end_i]
        #print(X.shape)
        pred = model(X)
        loss = loss_func(pred, y)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

That's it: we've created and trained a minimal neural network (in this case, a
logistic regression, since we have no hidden layers) entirely from scratch!

Let's check the loss and accuracy and compare those to what we got
earlier. We expect that the loss will have decreased and accuracy to
have increased, and they have.



In [48]:
print(loss_func(model(X), y), accuracy(model(X), y))

tensor(2.3026, grad_fn=<NllLossBackward>) tensor(0.0312)


Using torch.nn.functional
------------------------------

We will now refactor our code, so that it does the same thing as before, only
we'll start taking advantage of PyTorch's ``nn`` classes to make it more concise
and flexible. At each step from here, we should be making our code one or more
of: shorter, more understandable, and/or more flexible.

The first and easiest step is to make our code shorter by replacing our
hand-written activation and loss functions with those from ``torch.nn.functional``
(which is generally imported into the namespace ``F`` by convention). This module
contains all the functions in the ``torch.nn`` library (whereas other parts of the
library contain classes). As well as a wide range of loss and activation
functions, you'll also find here some convenient functions for creating neural
nets, such as pooling functions. (There are also functions for doing convolutions,
linear layers, etc, but as we'll see, these are usually better handled using
other parts of the library.)

If you're using negative log likelihood loss and log softmax activation,
then Pytorch provides a single function ``F.cross_entropy`` that combines
the two. So we can even remove the activation function from our model.



In [49]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(X):
    return X.reshape(-1, n_neurons).mm(weights) + bias

Note that we no longer call ``log_softmax`` in the ``model`` function. Let's
confirm that our loss and accuracy are the same as before:



In [50]:
print(loss_func(model(X), y), accuracy(model(X), y))

tensor(nan, grad_fn=<NllLossBackward>) tensor(0.1406)


# Refactor using nn.Module

Next up, we'll use ``nn.Module`` and ``nn.Parameter``, for a clearer and more concise training loop. 
We subclass ``nn.Module`` (which itself is a class and able to keep track of state). 
In this case, we want to create a class that holds our weights, bias, and method for the forward step.  
``nn.Module`` has a number of attributes and methods (such as ``.parameters()`` and ``.zero_grad()``) which we will be using.

#### Note

``nn.Module`` (uppercase M) is a PyTorch specific concept, and is a
class we'll be using a lot. ``nn.Module`` is not to be confused with the Python
concept of a (lowercase ``m``)  [module](https://docs.python.org/3/tutorial/modules.html), 
which is a file of Python code that can be imported.

In [51]:
from torch import nn

class my_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(n_neurons, n_classes) / math.sqrt(n_neurons))
        self.bias = nn.Parameter(torch.zeros(n_classes))

    def forward(self, X):
        return X.reshape(-1, n_neurons).mm(self.weights) + self.bias

Since we're now using an object instead of just using a function, we
first have to instantiate our model:



In [52]:
model = my_Logistic()

Now we can calculate the loss in the same way as before. Note that
``nn.Module`` objects are used as if they are functions (i.e they are
*callable*), but behind the scenes Pytorch will call our ``forward``
method automatically.



In [53]:
print(loss_func(model(X), y))

tensor(150.3418, grad_fn=<NllLossBackward>)


Previously for our training loop we had to update the values for each parameter
by name, and manually zero out the grads for each parameter separately, like this:

```python
with torch.no_grad():
    weights -= weights.grad * lr
    bias -= bias.grad * lr
    weights.grad.zero_()
    bias.grad.zero_()
```

Now we can take advantage of model.parameters() and model.zero_grad() (which
are both defined by PyTorch for ``nn.Module``) to make those steps more concise
and less prone to the error of forgetting some of our parameters, particularly
if we had a more complicated model:

```python
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```

We'll wrap our little training loop in a ``fit`` function so we can run it again later.

In [54]:
def fit():
    for epoch in range(epochs):
        for i in range((n_samples - 1) // batch_size + 1):
            start_i = i * batch_size
            end_i = start_i + batch_size
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

Let's double-check that our loss has gone down:



In [55]:
print(loss_func(model(X), y))

tensor(176.5281, grad_fn=<NllLossBackward>)


# Refactor using nn.Linear

We continue to refactor our code.  Instead of manually defining and
initializing ``self.weights`` and ``self.bias``, and calculating 
``xb  @ self.weights + self.bias``, 
we will instead use the Pytorch class [nn.Linear](https://pytorch.org/docs/stable/nn.html#linear-layers) for a linear layer, which does all that for us. Pytorch has many types of
predefined layers that can greatly simplify our code, and often makes it
faster too.



In [56]:
class my_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(n_neurons, n_classes)

    def forward(self, xb):
        return self.lin(xb.reshape(-1,n_neurons))

We instantiate our model and calculate the loss in the same way as before:



In [57]:
model = my_Logistic()
print(loss_func(model(X), y))

tensor(91.1086, grad_fn=<NllLossBackward>)


We are still able to use our same ``fit`` method as before.



In [58]:
fit()

print(loss_func(model(X), y))

tensor(622.3627, grad_fn=<NllLossBackward>)


# Refactor using optim

Pytorch also has a package with various optimization algorithms, ``torch.optim``.
We can use the ``step`` method from our optimizer to take a forward step, instead
of manually updating each parameter.

This will let us replace our previous manually coded optimization step:

```python
with torch.no_grad():
    for p in model.parameters(): p -= p.grad * lr
    model.zero_grad()
```

and instead use just:

```python
opt.step()
opt.zero_grad()
```

(``optim.zero_grad()`` resets the gradient to 0 and we need to call it before
computing the gradient for the next minibatch.)



In [59]:
from torch import optim

We'll define a little function to create our model and optimizer so we
can reuse it in the future.



In [60]:
def get_model():
    model = my_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(X), y))

for epoch in range(epochs):
    for i in range((n_samples - 1) // batch_size + 1):
        start_i = i * batch_size
        end_i = start_i + batch_size
        X = x_train[start_i:end_i]
        y = y_train[start_i:end_i]
        pred = model(X)
        loss = loss_func(pred, y)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(X), y))

tensor(74.5082, grad_fn=<NllLossBackward>)
tensor(3102.6333, grad_fn=<NllLossBackward>)


# Refactor using Dataset

PyTorch has an abstract Dataset class.  
A Dataset can be anything that has a ``__len__`` function (called by Python's standard ``len`` function) 
and a ``__getitem__`` function as a way of indexing into it.
[This tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) walks through a nice example of creating a custom ``FacialLandmarkDataset`` class as a subclass of ``Dataset``.

PyTorch's [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset) 
is a Dataset wrapping tensors. By defining a length and way of indexing,
this also gives us a way to iterate, index, and slice along the first
dimension of a tensor. This will make it easier to access both the
independent and dependent variables in the same line as we train.



In [61]:
from torch.utils.data import TensorDataset

Both ``x_train`` and ``y_train`` can be combined in a single ``TensorDataset``,
which will be easier to iterate over and slice.



In [62]:
train_ds = TensorDataset(x_train, y_train)

Previously, we had to iterate through minibatches of x and y values separately:
```python
X = x_train[start_i:end_i]
y = y_train[start_i:end_i]
```

Now, we can do these two steps together:
```python
X, y = train_ds[i*bs : i*bs+bs]
```



In [63]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n_samples - 1) // batch_size + 1):
        X, y = train_ds[i * batch_size: i * batch_size + batch_size]
        pred = model(X)
        loss = loss_func(pred, y)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(X), y))

tensor(5327.4536, grad_fn=<NllLossBackward>)


Refactor using DataLoader
------------------------------

Pytorch's ``DataLoader`` is responsible for managing batches. You can
create a ``DataLoader`` from any ``Dataset``. ``DataLoader`` makes it easier
to iterate over batches. Rather than having to use ``train_ds[i*bs : i*bs+bs]``,
the DataLoader gives us each minibatch automatically.



In [64]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size)

Previously, our loop iterated over batches (xb, yb) like this:

```python
for i in range((n-1)//batch_size + 1):
    xb,yb = train_ds[i*batch_size : i* batch_size + batch_size]
    pred = model(xb)
```

Now, our loop is much cleaner, as (xb, yb) are loaded automatically from the data loader:

```python
for xb,yb in train_dl:
    pred = model(xb)
```



In [65]:
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(6497.7241, grad_fn=<NllLossBackward>)


Thanks to Pytorch's ``nn.Module``, ``nn.Parameter``, ``Dataset``, and ``DataLoader``,
our training loop is now dramatically smaller and easier to understand. Let's
now try to add the basic features necessary to create effecive models in practice.

## Add validation

In section 1, we were just trying to get a reasonable training loop set up for
use on our training data.  In reality, you **always** should also have
a [validation set](https://www.fast.ai/2017/11/13/validation-sets/), in order to identify if you are overfitting.

Shuffling the training data is
[important](https://www.quora.com/Does-the-order-of-training-data-matter-when-training-neural-networks)
to prevent correlation between batches and overfitting. On the other hand, the
validation loss will be identical whether we shuffle the validation set or not.
Since shuffling takes extra time, it makes no sense to shuffle the validation data.

We'll use a batch size for the validation set that is twice as large as
that for the training set. This is because the validation set does not
need backpropagation and thus takes less memory (it doesn't need to
store the gradients). We take advantage of this to use a larger batch
size and compute the loss more quickly.



In [66]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)

We will calculate and print the validation loss at the end of each epoch.

(Note that we always call ``model.train()`` before training, and ``model.eval()``
before inference, because these are used by layers such as ``nn.BatchNorm2d``
and ``nn.Dropout`` to ensure appropriate behaviour for these different phases.)



In [67]:
model, opt = get_model()

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

0 tensor(48020.1680)
1 tensor(42782.1328)
2 tensor(45726.5742)
3 tensor(133182.0312)
4 tensor(35596.6250)


Create fit() and get_data()
----------------------------------

We'll now do a little refactoring of our own. Since we go through a similar
process twice of calculating the loss for both the training set and the
validation set, let's make that into its own function, ``loss_batch``, which
computes the loss for one batch.

We pass an optimizer in for the training set, and use it to perform
backprop.  For the validation set, we don't pass an optimizer, so the
method doesn't perform backprop.



In [68]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

``fit`` runs the necessary operations to train our model and compute the
training and validation losses for each epoch.



In [86]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

``get_data`` returns dataloaders for the training and validation sets.



In [87]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

Now, our whole process of obtaining the data loaders and fitting the
model can be run in 3 lines of code:



In [88]:
train_dl, valid_dl = get_data(train_ds, valid_ds, batch_size)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 12024.5721984375
1 10239.34226484375
2 9750.85490859375
3 8230.6672640625
4 9813.3264875


You can use these basic 3 lines of code to train a wide variety of models.
Let's see if we can use them to train a convolutional neural network (CNN)!

Switch to CNN
-------------

We are now going to build our neural network with three convolutional layers.
Because none of the functions in the previous section assume anything about
the model form, we'll be able to use them to train a CNN without any modification.

We will use Pytorch's predefined
`Conv2d <https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d>`_ class
as our convolutional layer. We define a CNN with 3 convolutional layers.
Each convolution is followed by a ReLU.  At the end, we perform an
average pooling.  (Note that ``view`` is PyTorch's version of numpy's
``reshape``)



In [89]:
class my_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

[Momentum](https://cs231n.github.io/neural-networks-3/#sgd) is a variation on stochastic gradient descent that takes previous updates into account as well and generally leads to faster training.



In [90]:
model = my_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.3025827449798584
1 2.3025827449798584
2 2.3025827449798584
3 2.3025827449798584
4 2.3025827449798584


nn.Sequential
------------------------

``torch.nn`` has another handy class we can use to simply our code:
[Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential). 
A ``Sequential`` object runs each of the modules contained within it, in a
sequential manner. This is a simpler way of writing our neural network.

To take advantage of this, we need to be able to easily define a
**custom layer** from a given function.  For instance, PyTorch doesn't
have a `view` layer, and we need to create one for our network. ``Lambda``
will create a layer that we can then use when defining a network with
``Sequential``.



In [74]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def preprocess(x):
    return x.view(-1, 1, 28, 28)

The model created with ``Sequential`` is simply:



In [75]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.3025827449798584
1 2.3025827449798584
2 2.3025827449798584
3 2.3025827449798584
4 2.3025827449798584


Wrapping DataLoader
-----------------------------

Our CNN is fairly concise, but it only works with MNIST, because:
 - It assumes the input is a 28\*28 long vector
 - It assumes that the final CNN grid size is 4\*4 (since that's the average
pooling kernel size we used)

Let's get rid of these two assumptions, so our model works with any 2d
single channel image. First, we can remove the initial Lambda layer but
moving the data preprocessing into a generator:



In [76]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, batch_size)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Next, we can replace ``nn.AvgPool2d`` with ``nn.AdaptiveAvgPool2d``, which
allows us to define the size of the *output* tensor we want, rather than
the *input* tensor we have. As a result, our model will work with any
size input.



In [77]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Let's try it out:



In [78]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.3025827449798584
1 2.3025827449798584
2 2.3025827449798584
3 2.3025827449798584
4 2.3025827449798584


Using your GPU
---------------

If you're lucky enough to have access to a CUDA-capable GPU (you can
rent one for about $0.50/hour from most cloud providers) you can
use it to speed up your code. First check that your GPU is working in
Pytorch:



In [79]:
print(torch.cuda.is_available())

True


And then create a device object for it:



In [80]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

Let's update ``preprocess`` to move batches to the GPU:



In [81]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)


train_dl, valid_dl = get_data(train_ds, valid_ds, batch_size)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Finally, we can move our model to the GPU.



In [82]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

You should find it runs faster now:



In [83]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

0 2.302584887313843
1 2.302584887313843
2 2.302584887313843
3 2.302584887313843
4 2.302584887313843


Closing thoughts
-----------------

We now have a general data pipeline and training loop which you can use for
training many types of models using Pytorch. To see how simple training a model
can now be, take a look at the `mnist_sample` sample notebook.

Of course, there are many things you'll want to add, such as data augmentation,
hyperparameter tuning, monitoring training, transfer learning, and so forth.
These features are available in the fastai library, which has been developed
using the same design approach shown in this tutorial, providing a natural
next step for practitioners looking to take their models further.

We promised at the start of this tutorial we'd explain through example each of
``torch.nn``, ``torch.optim``, ``Dataset``, and ``DataLoader``. So let's summarize
what we've seen:

 - **torch.nn**

   + ``Module``: creates a callable which behaves like a function, but can also
     contain state(such as neural net layer weights). It knows what ``Parameter`` (s) it
     contains and can zero all their gradients, loop through them for weight updates, etc.
   + ``Parameter``: a wrapper for a tensor that tells a ``Module`` that it has weights
     that need updating during backprop. Only tensors with the `requires_grad` attribute set are updated
   + ``functional``: a module(usually imported into the ``F`` namespace by convention)
     which contains activation functions, loss functions, etc, as well as non-stateful
     versions of layers such as convolutional and linear layers.
 - ``torch.optim``: Contains optimizers such as ``SGD``, which update the weights
   of ``Parameter`` during the backward step
 - ``Dataset``: An abstract interface of objects with a ``__len__`` and a ``__getitem__``,
   including classes provided with Pytorch such as ``TensorDataset``
 - ``DataLoader``: Takes any ``Dataset`` and creates an iterator which returns batches of data.

